In [24]:
from gen_finetune.run_finetune_experiment import get_dataset, prep_dataset
from pathlib import Path
import json


In [30]:
data_folder = Path("data/title_and_first_sen")
dataset = get_dataset(str(data_folder / "test_data"))
print(dataset)

(Dataset({
    features: ['task_input_a', 'task_input_b', 'task_answer_a', 'task_answer_b'],
    num_rows: 2613
}), TaskDescription(prompt_a='Which subreddit does this belong to? -----', prompt_b='Continue the story. -----', tag_a='reddit', tag_b='story'))
